# LCUMultiplexorBox Example
- PrepareBox implement with pytket `StatePreparationBox`
- SelectBox box implemented with pytket `MultiplexedTensoredU2Box`

In [1]:
from qtnmtts.circuits.lcu import LCUMultiplexorBox
from qtnmtts.operators import ising_model

## 3 qubit Ising model Hamiltonian

In [2]:
n_state_qubits = 3
h = 1
j = 1
ising_model_3q = ising_model(3, h, j)
print(ising_model_3q)

{(Zq[0], Zq[1]): 1.00000000000000, (Zq[1], Zq[2]): 1.00000000000000, (Xq[0]): 1.00000000000000, (Xq[1]): 1.00000000000000, (Xq[2]): 1.00000000000000}


Initialise LCUMultiplexorBox with:
- Operator (QubitPauliOperator)
- n_state_qubits (int)

In [3]:
lcu_box = LCUMultiplexorBox(ising_model_3q,n_state_qubits)

- In qtnmtts we should only work with `box.reg_circuit()`
- The underlying canonical circuit why pytket handle should never be exposed as we only need to work with register mapping


In [4]:
from pytket.circuit.display import render_circuit_jupyter
render_circuit_jupyter(lcu_box.reg_circuit)

Initialise `RegisterCircuit` with `initialise_circuit()`
- state qubits are labeled `q` register
- prepare qubits are labeled `p` register 

With qtnmtts we can work directly with registers and not worry about the canonical ordering of the circuit

In [5]:
circ = lcu_box.initialise_circuit()
print(circ.q_registers)

[QubitRegister("p", 3), QubitRegister("q", 3)]


In [6]:
from pytket import Qubit
circ.Ry(0.5,Qubit('q', 0)).Ry(0.75,Qubit('q', 1)).Ry(0.7,Qubit('q',2))

[Ry(0.5) q[0]; Ry(0.75) q[1]; Ry(0.7) q[2]; ]

Add oracle box to circuit with `add_oraclebox()`
- We can directly map the registers in the box to those in the circuit with `QRegMap`

In [7]:
from qtnmtts.circuits.core import QRegMap
qreg_map = QRegMap(lcu_box.q_registers, circ.q_registers)
circ.add_registerbox(lcu_box, qreg_map) 

[Ry(0.5) q[0]; Ry(0.75) q[1]; Ry(0.7) q[2]; CircBox p[0], p[1], p[2], q[0], q[1], q[2]; ]

In [8]:
render_circuit_jupyter(circ)

- We can do very useful debugging using the utils method 'circuit_unitary_postselect'
- This will pick out the sub block of the unitary corresponding to the qubits we post select on.

In [9]:
from  qtnmtts.measurement.utils import circuit_unitary_postselect

circuit_unitary_postselect(lcu_box.reg_circuit, lcu_box.postselect)

array([[ 4.00000000e-01+0.00000000e+00j,  2.00000000e-01+2.77555756e-17j,
         2.00000000e-01-2.91433544e-16j,  1.18129492e-16-8.07090191e-17j,
         2.00000000e-01-2.35922393e-16j,  5.59767304e-17+5.22036353e-17j,
        -1.58216172e-17-1.12293004e-16j,  7.61673575e-17+1.05959366e-17j],
       [ 2.00000000e-01+9.71445147e-17j,  8.63458486e-17-1.93859945e-16j,
        -1.10191564e-16-5.38306898e-17j,  2.00000000e-01-3.33066907e-16j,
        -1.93271134e-16+1.08877790e-16j,  2.00000000e-01+2.49800181e-16j,
        -5.43897683e-17+3.41285088e-17j, -2.37023971e-17-1.70939750e-16j],
       [ 2.00000000e-01-6.93889390e-17j,  5.20703725e-17+8.47990817e-18j,
        -4.00000000e-01-2.22044605e-16j,  2.00000000e-01+8.32667268e-17j,
         1.00548437e-16-2.11238678e-16j,  4.36834070e-17-3.12353232e-17j,
         2.00000000e-01+9.71445147e-17j, -3.67863138e-17-6.50578949e-18j],
       [-1.18823377e-16-2.44344860e-18j,  2.00000000e-01-2.08166817e-16j,
         2.00000000e-01+1.66533454e